In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    print("✅ Gemini API key loaded successfully.")
except Exception as e:
    print(f"❌ Authentication Error. Details: {e}")

✅ Gemini API key loaded successfully.


In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, FunctionTool
from google.genai import types

print("✅ Google ADK modules imported successfully.")

✅ Google ADK modules imported successfully.


In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# 1. Research and Summarization System

In [5]:
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model='gemini-2.5-flash-lite',
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",
)
print("✅ ResearchAgent created successfully.")

✅ ResearchAgent created successfully.


In [ ]:
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}
    Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ SummarizerAgent created.")

✅ SummarizerAgent created.


In [8]:
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a research coordinator. Your job is to answer the user's query by orchestrating a workflow.
    1. First, you MUST call the 'ResearchAgent' tool to find relevant information on the topic provided by the user.
    2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
    3. Finally, present the final summary clearly to the user as your response.""",
    tools=[
        AgentTool(research_agent), AgentTool(summarizer_agent)],
)
print("✅ Root Agent created.")

✅ Root Agent created.


In [11]:
runner = InMemoryRunner(agent=root_agent, app_name="agents")
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?
ResearchCoordinator > Recent advancements in quantum computing are poised to significantly impact Artificial Intelligence (AI), creating a future of \"Quantum AI\" with the potential to solve complex problems currently beyond classical computers' reach.

Key takeaways include:

*   **Exponential Computational Boost:** Quantum computers, leveraging qubits and superposition, offer unprecedented computational power, drastically reducing AI model training times and energy consumption. Google's \"Willow\" chip exemplifies this with its remarkable speed in complex calculations.
*   **Revolutionized Problem-Solving:** Quantum computing\'s unique mathematical capabilities enable AI to tackle previously intractable problems, from accelerating drug discovery through molecular simulations to transforming financial risk assessment and portfolio optimization.
*   *